### USCRN Data: High-Octane Scraping

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import yaml 
import re
import itertools
from datetime import datetime

with open ("sources.yaml", "r") as yaml_file:
  sources = yaml.load(yaml_file, Loader=yaml.FullLoader)

##### 1.) Scrape Column Headers and Descriptions 

In [7]:
header_url = sources['USCRN']['headers']
header_response = requests.get(header_url)
header_soup = BeautifulSoup(header_response.content, "html.parser")

columns = str(header_soup).split("\n")[1].strip(" ").split(" ")
columns = list(map(lambda x: str.lower(x), columns)) # columns = [str.lower(c) for c in columns] -- faster?
columns.insert(0,'station_location')

descrip_text = str(header_soup).split("\n")[2] # raw text block containing column descriptions
descrip_text

"The station WBAN number. The UTC date of the observation. The UTC time of the observation. Time is the end of the observed hour, so the 0000 hour is actually the last hour of the previous day's observation (starting just after 11:00 PM through midnight). The Local Standard Time (LST) date of the observation. The Local Standard Time (LST) time of the observation. Time is the end of the observed hour (see UTC_TIME description). The version number of the station datalogger program that was in effect at the time of the observation. Note: This field should be treated as text (i.e. string). Station longitude, using WGS-84. Station latitude, using WGS-84. Average air temperature, in degrees C, during the last 5 minutes of the hour. See Note F. Average air temperature, in degrees C, for the entire hour. See Note F. Maximum air temperature, in degrees C, during the hour. See Note F. Minimum air temperature, in degrees C, during the hour. See Note F. Total amount of precipitation, in mm, record

The descriptions of the columns are quite the mess, as there is no standard separator used. We will have to work our way through it step by step: 

In [3]:
first_split = [re.sub(r'(\([^)]*)$', r"\1)", s) for s in descrip_text.split("). ")] # add ')' back after splitting text on ').' 
no_notes = [re.sub(r' See Note [A-Z]\.',"",s) for s in first_split] # drop any references to notes

The third entry in `no_notes` is ready. The last set of descriptions in `no_notes` can be split on `". "`, but the first two sets need special attention. We will pop the last set out and split it, then pop the third set out, and then address the first two sets. At that point we will recombine everything into one list while preserving the original order. 

In [4]:
last_set = no_notes.pop().strip().split(". ")
third_set = no_notes.pop() # Note: just a string

In [5]:
def flatten(ls:list): 
  return list(itertools.chain.from_iterable(ls)) 

no_notes = [re.sub(". Time is", " at", s) for s in no_notes]
first_second = flatten([s.split(". ") for s in no_notes])

# Finally:
descriptions = flatten([["Location name for USCRN station"], first_second, [third_set], last_set]) # Description added for "station_location" 

In [6]:
header_info = {
  'col_name': columns,
  'description': descriptions, 
  'units': ["X...(Various Lengths)","XXXXX", "YYYYMMDD", "HHmm", "YYYYMMDD", "HHmm", "XXXXXX", "Decimal_degrees", "Decimal_degrees", "Celsius", "Celsius", "Celsius", "Celsius", "mm", "W/m^2", "X", "W/m^2", "X", "W/m^2", "X", "X", "Celsius", "X", "Celsius", "X", "Celsius", "X", "%", "X", "m^3/m^3", "m^3/m^3", "m^3/m^3", "m^3/m^3", "m^3/m^3", "Celsius", "Celsius", "Celsius", "Celsius", "Celsius"]
}

header_df = pd.DataFrame(header_info)
# header_df.to_csv("data/column_descriptions.csv", index=True)

##### 2.) Scrape Core Data Files (>2 million rows)

In [3]:
base_url = sources["USCRN"]["index"]
base_response = requests.get(base_url)
base_soup = BeautifulSoup(base_response.content, "html.parser")

In [4]:
links = base_soup.find_all("a") # 'links' in this notebook will refer to <a> elements, not urls
years = [str(x).zfill(1) for x in range(2000,2024)]
year_links = [link for link in links if link['href'].rstrip('/') in years]

file_urls = []
for year_link in year_links: 
  year_url = base_url + year_link.get("href")
  response = requests.get(year_url) 
  soup = BeautifulSoup(response.content, 'html.parser')
  file_links = soup.find_all('a', href=re.compile(r'AK.*\.txt'))
  if file_links:
    new_file_urls = [year_url + link.getText() for link in file_links]
    file_urls.extend(new_file_urls)

In [5]:
rows = []
regex = r"([St.]*[A-Z][a-z]+_*[A-Za-z]*).*.txt" 
for url in file_urls:
  # Get location from url -- will add to BS results in next step
  file_name = re.search(regex, url).group(0)
  station_location = re.sub("(_formerly_Barrow.*|_[0-9].*)", "", file_name)
  # Get results 
  response = requests.get(url)
  soup = BeautifulSoup(response.content,'html.parser')
  soup_lines = [station_location + " " + line for line in str(soup).strip().split("\n")]
  #   soup_lines = [station_location + " " + line for line in str(soup).split("\n")] <-- Old Code 
  new_rows = [re.split('\s+', row) for row in soup_lines]
  rows.extend(new_rows)

In [8]:
df = pd.DataFrame(rows, columns=columns)

In [9]:
df.sample(10)

,station_location,wbanno,utc_date,utc_time,lst_date,lst_time,crx_vn,longitude,latitude,t_calc,...,soil_moisture_5,soil_moisture_10,soil_moisture_20,soil_moisture_50,soil_moisture_100,soil_temp_5,soil_temp_10,soil_temp_20,soil_temp_50,soil_temp_100
825018,Denali,96408,20160122,0600,20160121,2100,-9.000,-150.87,63.45,-9999.0,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1069058,Red_Dog_Mine,26655,20170203,0900,20170203,0000,2.424,-162.92,68.03,-7.9,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1716189,Toolik_Lake,96409,20201211,0200,20201210,1700,2.514,-149.40,68.65,-25.6,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1804470,Ivotuk,26564,20210106,1100,20210106,0200,2.515,-155.75,68.49,-35.0,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
911645,Red_Dog_Mine,26655,20161202,1700,20161202,0800,2.424,-162.92,68.03,-20.8,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
198496,St._Paul,25711,20090609,1000,20090609,0100,1.302,-170.21,57.16,3.6,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2127366,Utqiagvik,27516,20221116,1100,20221116,0200,2.424,-156.61,71.32,-5.0,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
255036,St._Paul,25711,20100315,0200,20100314,1700,1.302,-170.21,57.16,-14.4,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2120239,Utqiagvik,27516,20220123,1200,20220123,0300,2.424,-156.61,71.32,-18.5,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
787573,Sitka,25379,20151015,0100,20151014,1600,2.424,-135.33,57.06,8.0,...,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


In [10]:
# Check for NaNs
df[df.isna().any(axis=1)]

,station_location,wbanno,utc_date,utc_time,lst_date,lst_time,crx_vn,longitude,latitude,t_calc,...,soil_moisture_5,soil_moisture_10,soil_moisture_20,soil_moisture_50,soil_moisture_100,soil_temp_5,soil_temp_10,soil_temp_20,soil_temp_50,soil_temp_100


Let's drop the soil temperature and moisture columns. These fields have missing data (-99.0 or -9999.0) for the vast majority of records.

In [11]:
soil_columns = df.filter(regex="soil")
soil_columns[soil_columns.isin([-99.0, -9999.0]).any(axis=1)]
soil_columns

,soil_moisture_5,soil_moisture_10,soil_moisture_20,soil_moisture_50,soil_moisture_100,soil_temp_5,soil_temp_10,soil_temp_20,soil_temp_50,soil_temp_100
0,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
1,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
3,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
4,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
...,...,...,...,...,...,...,...,...,...,...
2163427,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2163428,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2163429,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0
2163430,-99.000,-99.000,-99.000,-99.000,-99.000,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0


In [12]:
df = df.filter(regex="^((?!soil).)*$")

Lastly, let's convert the date and time columns to `datetime` objects...

In [13]:
def combine_datetime(df):
    # pad time data with leading zeros
    df['utc_time'] = df['utc_time'].astype(str).str.pad(4, fillchar='0')
    df['lst_time'] = df['lst_time'].astype(str).str.pad(4, fillchar='0')

    # create new columns with concatenated datetime values
    df['utc_datetime'] = pd.to_datetime(df['utc_date'].astype(str) + df['utc_time'].fillna(''), format='%Y%m%d%H%M')
    df['lst_datetime'] = pd.to_datetime(df['lst_date'].astype(str) + df['lst_time'].fillna(''), format='%Y%m%d%H%M')

    # drop the original columns
    df.drop(['utc_date', 'utc_time', 'lst_date', 'lst_time'], axis=1, inplace=True)

    # reorder columns 
    cols = ['station_location', 'wbanno', 'utc_datetime', 'lst_datetime'] + list(df.columns)[4:-2]
    df = df[cols]

    return df

combine_datetime(df)

,station_location,wbanno,utc_datetime,lst_datetime,latitude,t_calc,t_hr_avg,t_max,t_min,p_calc,...,solarad_min_flag,sur_temp_type,sur_temp,sur_temp_flag,sur_temp_max,sur_temp_max_flag,sur_temp_min,sur_temp_min_flag,rh_hr_avg,rh_hr_avg_flag
0,Fairbanks,26494,2002-08-09 22:00:00,2002-08-09 13:00:00,64.97,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,...,0,R,19.4,3,-9999.0,0,-9999.0,0,0,3
1,Fairbanks,26494,2002-08-09 23:00:00,2002-08-09 14:00:00,64.97,10.0,11.1,12.0,10.0,-9999.0,...,0,R,14.6,0,-9999.0,0,-9999.0,0,0,3
2,Fairbanks,26494,2002-08-10 00:00:00,2002-08-09 15:00:00,64.97,12.0,10.5,12.0,9.9,0.0,...,0,R,15.0,0,-9999.0,0,-9999.0,0,0,3
3,Fairbanks,26494,2002-08-10 01:00:00,2002-08-09 16:00:00,64.97,12.1,11.9,12.2,11.5,0.0,...,0,R,15.4,0,-9999.0,0,-9999.0,0,0,3
4,Fairbanks,26494,2002-08-10 02:00:00,2002-08-09 17:00:00,64.97,11.9,12.0,12.1,11.9,0.0,...,0,R,14.2,0,-9999.0,0,-9999.0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2163427,Yakutat,25382,2023-02-17 09:00:00,2023-02-17 00:00:00,59.51,-0.2,-0.2,-0.1,-0.3,0.3,...,0,C,-1.9,0,-1.2,0,-2.3,0,95,0
2163428,Yakutat,25382,2023-02-17 10:00:00,2023-02-17 01:00:00,59.51,-0.2,-0.2,-0.1,-0.2,0.0,...,0,C,-1.2,0,-1.0,0,-1.8,0,95,0
2163429,Yakutat,25382,2023-02-17 11:00:00,2023-02-17 02:00:00,59.51,-0.3,-0.3,-0.2,-0.3,0.0,...,0,C,-1.8,0,-1.7,0,-2.0,0,95,0
2163430,Yakutat,25382,2023-02-17 12:00:00,2023-02-17 03:00:00,59.51,-0.3,-0.3,-0.3,-0.3,0.0,...,0,C,-1.8,0,-1.7,0,-1.9,0,95,0


...and add a `date_added` column: 

In [14]:
df['date_added_utc'] = datetime.utcnow()
# df.to_csv("data/uscrn.csv", index=False)

In [15]:
df.to_csv("data/uscrn.csv", index=False)

Let's also make a table for our various station locations. This will be useful when searching for the four-day forecasts in the NWS notebook. 

In [1]:
locations = df[['station_location', 'wbanno', 'longitude', 'latitude']].drop_duplicates()
# locations.to_csv("data/locations.csv", index=False)

##### 3.) Upload Core Data to BigQuery 

In [79]:
%%bash
bq mk -d --location=us-east4 team-week3:alaska

Dataset 'team-week3:alaska' successfully created.


Core Data: 

In [3]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Manually set schema to make columns required (NaN detection on upload) and avoid parsing certain columns as numeric (e.g. crx_vn, the flag columns)
# Setting these columns as strings will indicate that they are not made to have arithmetic calculations done on them
schema = [
  bigquery.SchemaField("station_location", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("wbanno", "STRING", mode="REQUIRED"),  
  bigquery.SchemaField("utc_datetime", "DATETIME", mode="REQUIRED"), 
  bigquery.SchemaField("lst_datetime", "DATETIME", mode="REQUIRED"), 
  bigquery.SchemaField("crx_vn", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("longitude", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("latitude", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("t_calc", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("t_hr_avg", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("t_max", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("t_min", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("p_calc", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("solarad", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("solarad_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("solarad_max", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("solarad_max_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("solarad_min", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("solarad_min_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp_type", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp_max", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp_max_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp_min", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("sur_temp_min_flag", "STRING", mode="REQUIRED"), 
  bigquery.SchemaField("rh_hr_avg", "FLOAT", mode="REQUIRED"), 
  bigquery.SchemaField("rh_hr_avg_flag", "STRING", mode="REQUIRED"), 
]

In [4]:
key_path = "/home/alex/.creds/alex-sa-tw3.json"
credentials = service_account.Credentials.from_service_account_file(
   key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id)

table_id = f"{credentials.project_id}.alaska.uscrn"

jc = bigquery.LoadJobConfig(
   source_format = bigquery.SourceFormat.CSV,
   autodetect=False,
   schema=schema,
   create_disposition="CREATE_IF_NEEDED",
   write_disposition="WRITE_TRUNCATE", 
   destination_table_description="Historical weather data from USCRN stations in Alaska"
)

job = client.load_table_from_dataframe(df, table_id, job_config=jc)

job.result()

LoadJob<project=team-week3, location=us-east4, id=5a820e52-394a-4930-8489-45da5cf3036d>

##### 4.) DAG Task for Updating Dataset  

In [64]:
import datetime as dt 

now = dt.datetime.now()
updates_url = sources['USCRN']['updates'] + f"{now.year}"

df = pd.read_html("https://www.ncei.noaa.gov/pub/data/uscrn/products/hourly02/updates/2023/", skiprows=[1,2])[0]
df.drop(["Size", "Description"], axis=1, inplace=True)
df.dropna(inplace=True)
cols = [re.sub(" ","_",str.lower(c)) for c in df.columns]
df.columns = cols
df['last_modified'] = pd.to_datetime(df['last_modified'])
df

,name,last_modified
0,CRN60H0203-202301010100.txt,2022-12-31 20:47:00
1,CRN60H0203-202301010200.txt,2022-12-31 21:47:00
2,CRN60H0203-202301010300.txt,2022-12-31 22:53:00
3,CRN60H0203-202301010400.txt,2022-12-31 23:54:00
4,CRN60H0203-202301010500.txt,2023-01-01 00:49:00
...,...,...
1137,CRN60H0203-202302171000.txt,2023-02-17 05:47:00
1138,CRN60H0203-202302171100.txt,2023-02-17 06:47:00
1139,CRN60H0203-202302171200.txt,2023-02-17 07:47:00
1140,CRN60H0203-202302171300.txt,2023-02-17 08:47:00


In [83]:
from collections import deque
from io import StringIO

with open("data/uscrn.csv", 'r') as fp:
    q = deque(fp, 1)  
latest_date = pd.read_csv(StringIO(''.join(q)), header=None).iloc[0,3]

'2023-02-16 07:00:00'

In [28]:

cur_year = dt.datetime.now().year
latest_updates_link = soup.find(lambda tag: tag.name == "td" and f"{cur_year}" in tag.text)
latest_updates_link

<td><a href="2023/">2023/</a></td>

In [10]:
years = [str(x).zfill(1) for x in range(2000,2024)]
year_links = [link for link in links if link['href'].rstrip('/') in years]



<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">

<html>
<head>
<title>Index of /pub/data/uscrn/products/hourly02</title>
</head>
<body>
<h1>Index of /pub/data/uscrn/products/hourly02</h1>
<table>
<tr><th><a href="?C=N;O=D">Name</a></th><th><a href="?C=M;O=A">Last modified</a></th><th><a href="?C=S;O=A">Size</a></th><th><a href="?C=D;O=A">Description</a></th></tr>
<tr><th colspan="4"><hr/></th></tr>
<tr><td><a href="/pub/data/uscrn/products/">Parent Directory</a></td><td> </td><td align="right">  - </td><td> </td></tr>
<tr><td><a href="2000/">2000/</a></td><td align="right">2020-10-02 10:29  </td><td align="right">  - </td><td> </td></tr>
<tr><td><a href="2001/">2001/</a></td><td align="right">2020-10-02 10:29  </td><td align="right">  - </td><td> </td></tr>
<tr><td><a href="2002/">2002/</a></td><td align="right">2020-10-02 10:29  </td><td align="right">  - </td><td> </td></tr>
<tr><td><a href="2003/">2003/</a></td><td align="right">2020-10-02 10:29  </td><td align="right">  - <

In [ ]:
pd.read_csv()